In [17]:
# Algorithm 1: Four-query Clifford Tester
# https://www.arxiv.org/abs/2510.07164 pg. 21

import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator

In [18]:
from qiskit.circuit import Gate

def get_weyl_operator(a: list, b: list) -> Gate:
    """
    Create a gate implementing Weyl operator P_{a,b}.
    
    P_{a,b} = i^{<a,b>} Z^{a_1} ⊗ ... ⊗ Z^{a_n} X^{b_1} ⊗ ... ⊗ X^{b_n}
    
    The global phase i^{<a,b>} doesn't affect measurement outcomes, so we omit it.
    
    Args:
        a: List of n bits controlling Z gates
        b: List of n bits controlling X gates
        
    Returns:
        Gate implementing P_{a,b}
    """
    assert len(a) == len(b)

    qc = QuantumCircuit(len(a))
    
    # Apply Z gates where a_i = 1
    for i, ai in enumerate(a):
        if ai:
            qc.z(i)
    
    # Apply X gates where b_i = 1
    for i, bi in enumerate(b):
        if bi:
            qc.x(i)
    
    return qc.to_gate(label=f"P_{a},{b}")

In [19]:
def maximally_entangled_state(n: int) -> Gate:
    """
    Create a gate that prepares the maximally entangled state (tensor product of Bell pairs):
    |ψ⟩ = (1/√n) Σ_i |i⟩|i⟩
    
    This is done by applying H then CNOT to each pair of qubits.
    
    Args:
        n: Number of qubit pairs (total gate acts on 2n qubits)
        
    Returns:
        Gate that prepares the maximally entangled state on 2n qubits
    """
    qc = QuantumCircuit(2 * n)
    
    for i in range(n):
        qc.h(i)
        qc.cx(i, n + i)
    
    return qc.to_gate(label="Bell")

In [ ]:
def weyl_choi_state(n: int, x: list) -> Gate:
    """
    Create a gate that prepares the Choi state of Weyl operator P_x: |P_x⟩⟩
    
    This is done by:
    1. Preparing the maximally entangled state |ψ⟩
    2. Applying (P_x ⊗ I)|ψ⟩
    
    Args:
        n: Number of qubits per register (gate acts on 2n qubits)
        x: 2n-bit string, split as x = (a, b) where a controls Z, b controls X
        
    Returns:
        Gate that prepares |P_x⟩⟩ on 2n qubits (first n = register A, last n = register B)
    """
    assert len(x) == 2 * n
    
    a = x[:n]  # First n bits control Z gates
    b = x[n:]  # Last n bits control X gates
    
    qc = QuantumCircuit(2 * n)
    
    # Step 1: Prepare maximally entangled state
    bell = maximally_entangled_state(n)
    qc.append(bell, range(2 * n))
    
    # Step 2: Apply P_x to the first register only
    P_x = get_weyl_operator(a, b)
    qc.append(P_x, range(n))
    
    return qc.to_gate(label=f"|P_x⟩⟩")

In [21]:
def measure_bell_basis(qc: QuantumCircuit, qubits_A: list, qubits_B: list, clbits: list):
    """
    Measure in the Bell basis {|P_y⟩⟩⟨⟨P_y|}_y
    
    To measure in the Bell basis, we undo the Bell state preparation
    (CNOT then H) and measure in the computational basis.
    
    The measurement outcome directly gives us y = (a, b) where:
    - The B register measurement gives 'a' (which controlled Z)
    - The A register measurement gives 'b' (which controlled X)
    
    Args:
        qc: Quantum circuit to modify
        qubits_A: First register (n qubits)
        qubits_B: Second register (n qubits)
        clbits: Classical bits to store measurement results (2n bits)
    """
    n = len(qubits_A)
    assert len(qubits_B) == n
    assert len(clbits) == 2 * n
    
    # Undo Bell state preparation: CNOT then H
    for i in range(n):
        qc.cx(qubits_A[i], qubits_B[i])
        qc.h(qubits_A[i])
    
    # Measure: B register gives 'a', A register gives 'b'
    # Store as y = (a, b) = (B measurement, A measurement)
    for i in range(n):
        qc.measure(qubits_B[i], clbits[i])      # a bits
        qc.measure(qubits_A[i], clbits[n + i])  # b bits

In [ ]:
def clifford_tester(U_circuit: QuantumCircuit, n: int, shots: int = 1000):
    """
    Four-query Clifford tester algorithm.
    
    Tests whether a unitary U is a Clifford gate by:
    1. Sampling random x from F_2^{2n}
    2. Preparing two copies of U^{⊗2}|P_x⟩⟩
    3. Measuring each in the Bell basis to get y and y'
    4. Accepting if y = y'
    
    Args:
        U_circuit: A quantum circuit implementing the n-qubit unitary U
        n: Number of qubits U acts on
        shots: Number of times to run the test
        
    Returns:
        acceptance_rate: Fraction of runs where y = y'
    """
    accepts = 0
    simulator = AerSimulator()
    
    for _ in range(shots):
        # Step 1: Sample random x from F_2^{2n}
        x = list(np.random.randint(0, 2, size=2*n))
        
        # We need 4n qubits total:
        # |P_x⟩⟩ is a 2n-qubit state (Choi state of n-qubit operator)
        # U^{⊗2}|P_x⟩⟩ applies U to both halves of the Choi state
        # We need two independent copies of this, so 4n qubits total
        
        qc = QuantumCircuit(4*n, 4*n)
        
        # Qubit layout:
        # Copy 1: qubits 0 to n-1 (A1), qubits n to 2n-1 (B1)
        # Copy 2: qubits 2n to 3n-1 (A2), qubits 3n to 4n-1 (B2)
        A1 = list(range(0, n))
        B1 = list(range(n, 2*n))
        A2 = list(range(2*n, 3*n))
        B2 = list(range(3*n, 4*n))
        
        # Step 2a: Prepare |P_x⟩⟩ for both copies
        choi = weyl_choi_state(n, x)
        qc.append(choi, A1 + B1)
        qc.append(choi, A2 + B2)
        
        qc.barrier()
        
        # Step 2c: Apply U^{⊗2} = U ⊗ U to each copy
        # This means applying U to A1, U to B1, U to A2, U to B2
        for qubits in [A1, B1, A2, B2]:
            qc.compose(U_circuit, qubits=qubits, inplace=True)
        
        qc.barrier()
        
        # Step 3: Measure each copy in Bell basis
        c1 = list(range(0, 2*n))      # Classical bits for copy 1
        c2 = list(range(2*n, 4*n))    # Classical bits for copy 2
        
        measure_bell_basis(qc, A1, B1, c1)
        measure_bell_basis(qc, A2, B2, c2)
        
        # Decompose custom gates so AerSimulator can run them
        qc = qc.decompose()
        
        # Run the circuit
        result = simulator.run(qc, shots=1).result()
        counts = result.get_counts()
        
        # Get the measurement outcome (should be just one since shots=1)
        outcome = list(counts.keys())[0]
        
        # Split into y and y' (Qiskit returns bits in reverse order)
        # outcome is a 4n-bit string, first 2n bits are c2, last 2n bits are c1
        y_prime = outcome[:2*n]   # Copy 2 result
        y = outcome[2*n:]         # Copy 1 result
        
        # Step 4-5: Accept if y = y'
        if y == y_prime:
            accepts += 1
    
    return accepts / shots

In [23]:
# Test with Clifford gates (should accept with high probability)
print("Testing Clifford gates:")

# Hadamard (Clifford)
H = QuantumCircuit(1)
H.h(0)
print(f"  Hadamard acceptance rate: {clifford_tester(H, 1, shots=100):.2f}")

# S gate (Clifford)
S = QuantumCircuit(1)
S.s(0)
print(f"  S gate acceptance rate: {clifford_tester(S, 1, shots=100):.2f}")

# CNOT (Clifford, 2-qubit)
CNOT = QuantumCircuit(2)
CNOT.cx(0, 1)
print(f"  CNOT acceptance rate: {clifford_tester(CNOT, 2, shots=100):.2f}")

# Identity (Clifford)
I = QuantumCircuit(1)
print(f"  Identity acceptance rate: {clifford_tester(I, 1, shots=100):.2f}")

Testing Clifford gates:
  Hadamard acceptance rate: 1.00
  S gate acceptance rate: 1.00
  CNOT acceptance rate: 1.00
  Identity acceptance rate: 1.00


In [24]:
# Test with non-Clifford gates (should reject with noticeable probability)
print("\nTesting non-Clifford gates:")

# T gate (non-Clifford)
T = QuantumCircuit(1)
T.t(0)
print(f"  T gate acceptance rate: {clifford_tester(T, 1, shots=100):.2f}")

# Rotation by arbitrary angle (non-Clifford)
Rx = QuantumCircuit(1)
Rx.rx(0.3, 0)  # Arbitrary rotation angle
print(f"  Rx(0.3) acceptance rate: {clifford_tester(Rx, 1, shots=100):.2f}")

# Toffoli (non-Clifford, 3-qubit)
Toffoli = QuantumCircuit(3)
Toffoli.ccx(0, 1, 2)
print(f"  Toffoli acceptance rate: {clifford_tester(Toffoli, 3, shots=100):.2f}")


Testing non-Clifford gates:
  T gate acceptance rate: 0.78
  Rx(0.3) acceptance rate: 0.85
  Toffoli acceptance rate: 0.29
